In [25]:
pip install PyYaml

Contents of 'legacy_users' Table:
   index first_name last_name date_of_birth                        company  \
0      0   Sigfried     Noack    1990-09-30             Heydrich Junitz KG   
1      1        Guy     Allen    1940-12-01                        Fox Ltd   
2      2      Harry  Lawrence    1995-08-02      Johnson, Jones and Harris   
3      3     Darren   Hussain    1972-09-23                    Wheeler LLC   
4      4      Garry     Stone    1952-12-20                     Warner Inc   
5      5      David    Torres    1949-08-12                   Yang-Stewart   
6      6       Anne    Morris    1952-11-10                 Hutchinson Inc   
7      7      Louis   Roberts    2006-08-05  Hamilton, Walters and Clayton   
8      8   Kathleen    Barlow    1959-11-13                   Ferguson Ltd   
9      9      Emily     Jones    1992-10-09                Serrano-Leblanc   

                  email_address  \
0             rudi79@winkler.de   
1  rhodesclifford@henderson.com   
2 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Connect with and upload data to the database
import yaml
from sqlalchemy import create_engine, inspect

class DatabaseConnector:
#initialize the db engine (makes connecting to other classes smoother)
    def __init__(self, creds_path):
        self.creds_path = creds_path
        self.engine = self.init_db_engine()

#Create a method read_db_creds this will read the credentials yaml file and return a dictionary of the credentials.
    def read_db_creds(self):
        with open(self.creds_path, 'r') as creds:
            credentials = yaml.safe_load(creds)
        return dict(credentials)

#read the credentials from the return of read_db_creds and initialise and return an sqlalchemy database engine
    def init_db_engine(self):
        credentials = self.read_db_creds()
        db_url = f"postgresql://{credentials['RDS_USER']}:{credentials['RDS_PASSWORD']}@{credentials['RDS_HOST']}:{credentials['RDS_PORT']}/{credentials['RDS_DATABASE']}"
        engine = create_engine(db_url)
        return engine

#Using the engine from init_db_engine create a method list_db_tables to list all the tables in the database so you know which tables you can extract data from.
    def list_db_tables(self):
        inspector = inspect(self.engine)
        table_names = inspector.get_table_names()
        return table_names

In [ ]:
#This class will work as a utility class, in it you will be creating methods that help extract data from different data sources.
#The methods contained will be fit to extract data from a particular data source, these sources will include CSV files, an API and an S3 bucket.
import pandas as pd
from database_utils import DatabaseConnector

class DataExtractor:
#Initialize an instance via DatabaseConnector:
    def __init__(self):
        self.connector_instance = DatabaseConnector('/Users/itsanya/AiCore/MRDC/db_creds.yaml')

#Take in an instance of your DatabaseConnector class and the table name as an argument and return a pandas DataFrame
    def read_rds_table(self, table_name):
        engine = self.connector_instance.engine
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        return df

In [ ]:
#Get table names from the db
creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'
connector = DatabaseConnector(creds_path)
table_names = connector.list_db_tables()
print(table_names)

### Viewing the user data tables

Table 1: legacy_store_details

In [ ]:
#This class will work as a utility class, in it you will be creating methods that help extract data from different data sources.
#The methods contained will be fit to extract data from a particular data source, these sources will include CSV files, an API and an S3 bucket.
import pandas as pd
from database_utils import DatabaseConnector

class DataExtractor:
#Initialize an instance via DatabaseConnector:
    def __init__(self):
        self.connector_instance = DatabaseConnector('/Users/itsanya/AiCore/MRDC/db_creds.yaml')

#Take in an instance of your DatabaseConnector class and the table name as an argument and return a pandas DataFrame
    def read_rds_table(self, table_name):
        engine = self.connector_instance.engine
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        return df
    
    def get_user_data_table(self, connector_instance):
    # Obtain list of tables from DatabaseConnector instance
        tables = connector_instance.list_db_tables()

        user_data_table = None
        for table in tables:
            if 'users' in table.lower() or 'details' in table.lower() or 'tables' in table.lower():
                user_data_table = table
                break

        if user_data_table:
        # Extract table containing user data using read_rds_table method
            return self.read_rds_table(connector_instance, user_data_table)
        else:
            return None     

    def read_and_display_table(self, table_name):
        # Extract the DataFrame using read_rds_table method
        data_table = self.read_rds_table(table_name)

        # Display the DataFrame contents
        print(data_table.head(450)) 
        print(data_table.info()) 
        print(data_table.columns.tolist()) 
        print("Head of every column in the DataFrame:")
        for column in data_table.columns:
            print(f"Column: {column}")
            print(data_table[column].head())  # Display head of the column
            print("-----------------------")

data_extractor = DataExtractor()
data_extractor.read_and_display_table('legacy_store_details')

#replace \n with space, ensure objects, in address column
#ensure longitude is a float64 to 5dp
#remove lat due to None values only
#ensure object VARCHAR(100) for locality column
#ensure structure of store code values is: alpha char x2 - alphanumeric characters x8
#ensure staff numbers are int64
#ensure opening date column has correct timestamp structure
#ensure store type is object VARCHAR(100)
#ensure latitude is a float64 to 5dp
#ensure country code is an object that is VARCHAR(100).upper() - check code for this!
#ensure continent is is an object VARCHAR(100) - with first letter of each word as .upper()
#remove row 447!
#if all of the above are not complied with, show or remove those rows of data?

Table 2: legacy_users

In [ ]:
#This class will work as a utility class, in it you will be creating methods that help extract data from different data sources.
#The methods contained will be fit to extract data from a particular data source, these sources will include CSV files, an API and an S3 bucket.
import pandas as pd
from database_utils import DatabaseConnector

class DataExtractor:
#Initialize an instance via DatabaseConnector:
    def __init__(self):
        self.connector_instance = DatabaseConnector('/Users/itsanya/AiCore/MRDC/db_creds.yaml')

#Take in an instance of your DatabaseConnector class and the table name as an argument and return a pandas DataFrame
    def read_rds_table(self, table_name):
        engine = self.connector_instance.engine
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        return df
    
    def get_user_data_table(self, connector_instance):
    # Obtain list of tables from DatabaseConnector instance
        tables = connector_instance.list_db_tables()

        user_data_table = None
        for table in tables:
            if 'users' in table.lower() or 'details' in table.lower() or 'tables' in table.lower():
                user_data_table = table
                break

        if user_data_table:
        # Extract table containing user data using read_rds_table method
            return self.read_rds_table(connector_instance, user_data_table)
        else:
            return None     

    def read_and_display_table(self, table_name):
        # Extract the DataFrame using read_rds_table method
        data_table = self.read_rds_table(table_name)

        # Display the DataFrame contents
        print(f"Contents of '{table_name}' Table:")
        print(data_table.head(10)) 
        print(data_table.info())
        print(data_table.columns.tolist()) 
        print("Head of every column in the DataFrame:")
        for column in data_table.columns:
            print(f"Column: {column}")
            print(data_table[column].head())  # Display head of the column
            print("-----------------------")


data_extractor = DataExtractor()
data_extractor.read_and_display_table('legacy_users')

Table #: orders_table

In [ ]:
#This class will work as a utility class, in it you will be creating methods that help extract data from different data sources.
#The methods contained will be fit to extract data from a particular data source, these sources will include CSV files, an API and an S3 bucket.
import pandas as pd
from database_utils import DatabaseConnector

class DataExtractor:
#Initialize an instance via DatabaseConnector:
    def __init__(self):
        self.connector_instance = DatabaseConnector('/Users/itsanya/AiCore/MRDC/db_creds.yaml')

#Take in an instance of your DatabaseConnector class and the table name as an argument and return a pandas DataFrame
    def read_rds_table(self, table_name):
        engine = self.connector_instance.engine
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        return df
    
    def get_user_data_table(self, connector_instance):
    # Obtain list of tables from DatabaseConnector instance
        tables = connector_instance.list_db_tables()

        user_data_table = None
        for table in tables:
            if 'users' in table.lower() or 'details' in table.lower() or 'tables' in table.lower():
                user_data_table = table
                break

        if user_data_table:
        # Extract table containing user data using read_rds_table method
            return self.read_rds_table(connector_instance, user_data_table)
        else:
            return None     

    def read_and_display_table(self, table_name):
        # Extract the DataFrame using read_rds_table method
        data_table = self.read_rds_table(table_name)

        # Display the DataFrame contents
        print(f"Contents of '{table_name}' Table:")
        print(data_table.head(50)) 
        print(data_table.info())
        print(data_table.columns.tolist()) 
        print("Head of every column in the DataFrame:")
        for column in data_table.columns:
            print(f"Column: {column}")
            print(data_table[column].head())  # Display head of the column
            print("-----------------------")


data_extractor = DataExtractor()
data_extractor.read_and_display_table('orders_table')

#

All Steps:

In [ ]:
# Import necessary libraries
import yaml
import pandas as pd
from sqlalchemy import create_engine

# Define the DatabaseConnector class
class DatabaseConnector:
    def read_db_creds(self, creds_file='db_creds.yaml'):
        with open(creds_file, 'r') as file:
            credentials = yaml.safe_load(file)
        return credentials

    def init_db_engine(self):
        credentials = self.read_db_creds()
        db_string = f"postgresql+psycopg2://{credentials['RDS_USER']}:{credentials['RDS_PASSWORD']}@{credentials['RDS_HOST']}:{credentials['RDS_PORT']}/{credentials['RDS_DATABASE']}"
        engine = create_engine(db_string)
        return engine

    def list_db_tables(self):
        engine = self.init_db_engine()
        with engine.connect() as connection:
            tables = connection.execute(
                "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
            ).fetchall()
        return [table[0] for table in tables]

    def upload_to_db(self, df, table_name):
        engine = self.init_db_engine()
        df.to_sql(table_name, engine, if_exists='replace', index=False)


# Define the DataExtractor class
class DataExtractor:
    def read_rds_table(self, db_connector, table_name):
        engine = db_connector.init_db_engine()
        query = f"SELECT * FROM {table_name};"
        df = pd.read_sql(query, engine)
        return df


# Define the DataCleaning class
class DataCleaning:
    def clean_user_data(self, df):
        # Perform data cleaning operations here
        # For example: handle NULL values, correct data types, etc.
        # This method should clean the user data in the DataFrame
        cleaned_data = df.dropna()  # Placeholder example: dropping rows with NULL values
        return cleaned_data


# Example usage
if __name__ == "__main__":
    # Instantiate classes
    db_connector = DatabaseConnector()
    data_extractor = DataExtractor()
    data_cleaner = DataCleaning()

    # Step 4: List all tables in the database
    tables = db_connector.list_db_tables()
    print("Available tables:", tables)

    # Step 5: Read user data table into a DataFrame
    user_data_table = 'users_table_name'  # Replace with the actual table name
    user_data = data_extractor.read_rds_table(db_connector, user_data_table)

    # Step 6: Clean the user data
    cleaned_user_data = data_cleaner.clean_user_data(user_data)

    # Step 8: Upload cleaned data to the database
    db_connector.upload_to_db(cleaned_user_data, 'dim_users')

VIEW ENTIRE DF

In [ ]:
pip install pandasgui

In [ ]:
import os
os.environ['APPDATA'] = ""
import pandas as pd
from pandasgui import show
from data_extraction import DataExtractor

data_extractor = DataExtractor()
table_name = 'legacy_store_details'
df = data_extractor.read_rds_table(table_name)

show(df)



In [3]:
import pandas as pd
from sqlalchemy import create_engine
import os
os.environ['APPDATA'] = ""
import pandas as pd
from pandasgui import show

# Define the connection string using the provided credentials
RDS_HOST = 'data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com'
RDS_USER = 'aicore_admin'
RDS_PASSWORD = 'AiCore2022'
RDS_DATABASE = 'postgres'
RDS_PORT = 5432

# Create a connection string
connection = f'postgresql://{RDS_USER}:{RDS_PASSWORD}@{RDS_HOST}:{RDS_PORT}/{RDS_DATABASE}'

# Create a database engine
engine = create_engine(connection)

# Query your database to fetch data into pandas DataFrames
query1 = 'SELECT * FROM legacy_store_details'
query2 = 'SELECT * FROM legacy_users'
query3 = 'SELECT * FROM orders_table'

df1 = pd.read_sql_query(query1, engine)
df2 = pd.read_sql_query(query2, engine)
df3 = pd.read_sql_query(query3, engine)

# Close the database engine connection
engine.dispose()

show(df1)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


ValueError: cannot insert level_0, already exists

In [9]:
import yaml
from sqlalchemy import create_engine, inspect


class DatabaseConnector:
    def __init__(self, creds_path):
        self.creds_path = creds_path
        self.engine = self.init_db_engine()

    def read_db_creds(self):
        with open(self.creds_path, 'r') as creds:
            credentials = yaml.safe_load(creds)
        return dict(credentials)
    
    def init_db_engine(self):
        credentials = self.read_db_creds()
        db_url = f"postgresql://{credentials['RDS_USER']}:{credentials['RDS_PASSWORD']}@{credentials['RDS_HOST']}:{credentials['RDS_PORT']}/{credentials['RDS_DATABASE']}"
        engine = create_engine(db_url)
        return engine    

    def list_db_tables(self):
        inspector = inspect(self.engine)
        table_names = inspector.get_table_names()
        return table_names  
    

if __name__ == "__main__":
    # Initialize DatabaseConnector with credentials file path
    db_connector = DatabaseConnector('/Users/itsanya/AiCore/MRDC/db_creds.yaml')

    # Retrieve and print table names
    tables = db_connector.list_db_tables()
    for table in tables:
            print(table)


legacy_store_details
legacy_users
orders_table


## FULL CODE CHECKPOINT

In [22]:
import yaml
from sqlalchemy import create_engine, inspect


class DatabaseConnector:
    def __init__(self, creds_path):
        self.creds_path = creds_path
        self.engine = self.init_db_engine()

    def read_db_creds(self):
        with open(self.creds_path, 'r') as creds:
            credentials = yaml.safe_load(creds)
        return dict(credentials)
    
    def init_db_engine(self):
        credentials = self.read_db_creds()
        db_url = f"postgresql://{credentials['RDS_USER']}:{credentials['RDS_PASSWORD']}@{credentials['RDS_HOST']}:{credentials['RDS_PORT']}/{credentials['RDS_DATABASE']}"
        engine = create_engine(db_url)
        return engine    

    def list_db_tables(self):
        inspector = inspect(self.engine)
        table_names = inspector.get_table_names()
        return table_names    



import pandas as pd
from database_utils import DatabaseConnector

class DataExtractor:

    def __init__(self, database_connector):
        self.database_connector = database_connector
    
    def read_rds_table(self, table_name):
        tables = self.database_connector.list_db_tables()
        
        if table_name in tables:
            engine = self.database_connector.engine
            return pd.read_sql_table(table_name, con=engine)


creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'
db_connector_instance = DatabaseConnector(creds_path) 
data_extractor = DataExtractor(db_connector)  

table_to_extract = 'legacy_store_details'
legacy_store_details_df = data_extractor.read_rds_table(table_to_extract)
print('Legacy Store Details Table\n', legacy_store_details_df.head())
print('\n')

table_to_extract = 'legacy_users'
legacy_users_df = data_extractor.read_rds_table(table_to_extract)
print('Legacy Users Table\n', legacy_users_df.head())
print('\n')

table_to_extract = 'orders_table'
orders_table_df = data_extractor.read_rds_table(table_to_extract)
print('Orders Table Table\n', orders_table_df.head())
print('\n')


Legacy Store Details Table
    index                                            address longitude   lat  \
0      1  Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...  51.62907  None   
1      2        Heckerstraße 4/5\n50491 Säckingen, Landshut  48.52961  None   
2      3  5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury     51.26  None   
3      4  Studio 6\nStephen landing\nSouth Simon\nB77 2W...   53.0233  None   
4      5  Flat 92u\nChristian harbors\nPort Charlotte\nN...  53.38333  None   

       locality   store_code staff_numbers opening_date   store_type  \
0  High Wycombe  HI-9B97EE4E            34   1996-10-25        Local   
1      Landshut  LA-0772C7B9            92   2013-04-12  Super Store   
2      Westbury  WE-1DE82CEE            69   2014-01-02  Super Store   
3        Belper  BE-18074576            35   2019-09-09        Local   
4  Gainsborough  GA-CAD01AC2            36   1995-05-15        Local   

   latitude country_code continent  
0  -0.74934           GB   

In [29]:
legacy_store_details_df.head()

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,None,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
1,2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,None,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
2,3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,None,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
3,4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,None,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
4,5,Flat 92u\nChristian harbors\nPort Charlotte\nN...,53.38333,None,Gainsborough,GA-CAD01AC2,36,1995-05-15,Local,-0.76667,GB,Europe


In [27]:
legacy_users_df.head()

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [28]:
orders_table_df.head()

,level_0,index,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
0,0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,NaN,3
1,1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,NaN,2
2,2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,NaN,2
3,3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,NaN,2
4,4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,NaN,2
